In [270]:
import pandas as pd
import os
from datetime import date, timedelta
from metric_helper import read_helper
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

## Load Data

In [271]:
def read_recommendations(dt_list):
    
    df = pd.DataFrame()
    
    for dt in dt_list:
        
        df_temp = pd.DataFrame()
    
        if os.path.exists(f'./results/execution/default/zips_to_remediate/{dt}/'):
            df_rem = pd.read_parquet(f'./results/execution/default/zips_to_remediate/{dt}/dea_th_0.9_fc_switch_0.15_rec_count_999999.parquet')
            df_temp = pd.concat([df_temp, df_rem])

        if os.path.exists(f'./results/execution/default/zips_to_expand/{dt}/'):
            df_exp = pd.read_parquet(f'./results/execution/default/zips_to_expand/{dt}/dea_th_0.9_fc_switch_0.15_rec_count_999999.parquet')
            df_temp = pd.concat([df_temp, df_exp])
            
        df_temp['run_date'] = dt
        
        df = pd.concat([df_temp, df])
        
        
    return df

In [272]:
def read_metadata(dt_list):
    
    df = pd.DataFrame()
    
    for dt in dt_list:
    
        if os.path.exists(f'./results/execution/default/metadata/{dt}/zips_to_recommend.parquet'):
            df_temp = pd.read_parquet(f'./results/execution/default/metadata/{dt}/zips_to_recommend.parquet')
            df_temp['run_date'] = dt
            df = pd.concat([df, df_temp])
        
        
    return df

In [273]:
dt_list = ['2025-11-13'] # '2025-11-10','2025-11-11','2025-11-12','2025-11-13'

df = read_recommendations(dt_list) # recommendations
df_m = read_metadata(dt_list) # metadata

In [274]:
df.head(2)

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date
8044,95531,1,25.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.749669,1,2025-11-13
3706,60517,1,31.285714,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.488105,1,2025-11-13


In [275]:
df_m.head(2)

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
0,33625,1,0.009217,31.000000,0.972414,1.000000,ok,145.0,127.0,0.041463,0.03,ok,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
3,33647,1,0.018182,62.857143,0.988281,0.995536,ok,256.0,224.0,0.050122,0.03,ok,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13


In [276]:
df = df.fillna('None')
df_m = df_m.fillna('None')

## Recommendation Distribution

In [277]:
df.groupby(['run_date','active','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,final_recommendation,zip5
0,2025-11-13,1,deactivate,133


In [278]:
df.groupby(['run_date','active','current_recommendation','last_recommendation','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,current_recommendation,last_recommendation,final_recommendation,zip5
0,2025-11-13,1,deactivate,None,deactivate,133


In [279]:
df_m.groupby(['run_date','active','current_recommendation','last_recommendation','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,current_recommendation,last_recommendation,final_recommendation,zip5
0,2025-11-13,1,None,None,None,4
1,2025-11-13,1,deactivate,None,deactivate,133
2,2025-11-13,1,ok,None,None,3145


In [280]:
df_m.groupby(['run_date','active','reason_code_dea','reason_code_backlog','current_recommendation','last_recommendation','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,reason_code_dea,reason_code_backlog,current_recommendation,last_recommendation,final_recommendation,zip5
0,2025-11-13,1,None,None,None,None,None,4
1,2025-11-13,1,None,ok,ok,None,None,131
2,2025-11-13,1,deactivate,None,deactivate,None,deactivate,14
3,2025-11-13,1,deactivate,ok,deactivate,None,deactivate,119
4,2025-11-13,1,ok,None,ok,None,None,290
5,2025-11-13,1,ok,ok,ok,None,None,2724


## Zip by Zip Analysis

In [224]:
df = df.sort_values(['zip5','run_date'])
df_m = df_m.sort_values(['zip5','run_date'])

In [225]:
df['count'] = df.groupby('zip5')['run_date'].transform('nunique')

In [226]:
df.loc[df['count'] == 1].head()

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count
2687,07030,1,111.857143,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.054889,1,2025-11-10,1
2692,07036,1,40.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.167628,1,2025-11-10,1
3622,07090,1,56.428571,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.128901,1,2025-11-10,1
3340,07866,1,58.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.037290,1,2025-11-10,1
1892,07920,1,44.500000,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.020662,1,2025-11-13,1


In [227]:
df_m.loc[df_m['zip5'] == '07066'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
1125,07066,1,0.0,29.285714,0.989362,0.905172,ok,94.0,116.0,None,0.07,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-10
2933,07066,1,0.0,30.428571,1.000000,0.965217,ok,76.0,115.0,None,0.07,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-11
1194,07066,1,0.0,29.285714,1.000000,0.982609,ok,76.0,115.0,None,0.07,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-12
3051,07066,1,0.0,30.500000,1.000000,0.980392,ok,76.0,102.0,None,0.07,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13


In [228]:
df.loc[df['count'] == 2].head()

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count
2693,07039,1,39.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.018665,1,2025-11-10,2
2416,07039,1,40.857143,deactivate,deactivate,None,None,deactivate,deactivate,None,None,deactivate,deactivate,None,None,0.027142,1,2025-11-11,2
1023,10026,1,48.142857,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.172879,1,2025-11-10,2
4774,10026,1,45.285714,deactivate,deactivate,None,None,deactivate,deactivate,None,None,deactivate,deactivate,None,None,0.173637,1,2025-11-11,2
1024,10027,1,73.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.279515,1,2025-11-10,2


In [229]:
df_m.loc[df_m['zip5'] == '07042'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
1111,07042,1,0.004751,60.142857,0.993492,0.884615,deactivate,461.0,26.0,None,0.06,None,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-10
2922,07042,1,0.002532,56.428571,0.993617,0.88,deactivate,470.0,25.0,None,0.06,None,deactivate,deactivate,None,None,deactivate,2025-11-10,deactivate,None,None,deactivate,deactivate,None,None,2025-11-11
1180,07042,1,0.002890,49.428571,0.993407,0.88,deactivate,455.0,25.0,None,0.06,None,deactivate,deactivate,None,None,deactivate,2025-11-11,deactivate,None,None,deactivate,deactivate,None,None,2025-11-12
3037,07042,1,0.003571,46.666667,0.992208,0.88,deactivate,385.0,25.0,None,0.06,None,deactivate,deactivate,None,None,deactivate,2025-11-12,deactivate,None,None,deactivate,deactivate,None,None,2025-11-13


In [230]:
df.loc[df['count'] == 3].head()

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count
1755,07950,1,39.000000,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.099161,1,2025-11-11,3
3616,07950,1,36.857143,deactivate,deactivate,None,None,deactivate,deactivate,None,None,deactivate,deactivate,None,None,0.103267,1,2025-11-12,3
1900,07950,1,37.166667,deactivate,deactivate,None,None,deactivate,deactivate,None,None,deactivate,deactivate,None,None,0.107331,1,2025-11-13,3
5913,16506,1,35.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.184541,1,2025-11-11,3
1007,16506,1,32.571429,deactivate,deactivate,None,None,deactivate,deactivate,None,None,deactivate,deactivate,None,None,0.183506,1,2025-11-12,3


In [232]:
df_m.loc[df_m['zip5'] == '07950'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
3650,07950,1,0.0,39.714286,0.990854,0.916667,ok,328.0,12.0,None,0.06,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-10
1755,07950,1,0.0,39.000000,0.996865,0.857143,deactivate,319.0,14.0,None,0.06,None,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-11
3616,07950,1,0.0,36.857143,0.996753,0.846154,deactivate,308.0,13.0,None,0.06,None,deactivate,deactivate,None,None,deactivate,2025-11-11,deactivate,None,None,deactivate,deactivate,None,None,2025-11-12
1900,07950,1,0.0,37.166667,0.996296,0.833333,deactivate,270.0,12.0,None,0.06,None,deactivate,deactivate,None,None,deactivate,2025-11-12,deactivate,None,None,deactivate,deactivate,None,None,2025-11-13


## Particular Zip Analysis

### NA DEA zips

In [281]:
dea = read_helper(
    './data/execution_data/unpadded_dea',
    start_date='2025-10-31',
    end_date='2025-11-11',
    date_col_name='unpadded_edd',
    cols=['unpadded_edd',
        'ffmcenter_name',
        'carrier_code',
        'zip5',
        'package_count',
        'unpadded_edd_dea_count'
        ]
    )
dea.head(2)

,unpadded_edd,ffmcenter_name,carrier_code,zip5,package_count,unpadded_edd_dea_count
0,2025-10-31,AVP1,FDXHD,01225,3,3
1,2025-10-31,AVP1,FDXHD,01267,3,3


In [282]:
dea = dea.groupby(
    ['zip5','unpadded_edd', 'carrier_code']).agg(
        {'package_count':'sum', 'unpadded_edd_dea_count':'sum'}).reset_index()
dea = dea.sort_values(['zip5','unpadded_edd'])

In [283]:
dea.groupby(['carrier_code','unpadded_edd'])['zip5'].nunique()

carrier_code  unpadded_edd
FDXHD         2025-10-31        542
              2025-11-01      23225
              2025-11-02       8675
              2025-11-03      25181
              2025-11-04      24421
              2025-11-05      25450
              2025-11-06      25641
              2025-11-07      25288
              2025-11-08      23210
              2025-11-09       8768
              2025-11-10      25085
              2025-11-11      24273
ONTRGD        2025-10-31          7
              2025-11-01       4179
              2025-11-02       5360
              2025-11-03       6164
              2025-11-04       6038
              2025-11-05       5893
              2025-11-06       5715
              2025-11-07       5457
              2025-11-08       4414
              2025-11-09       5389
              2025-11-10       6178
              2025-11-11       6081
Name: zip5, dtype: int64

In [284]:
df_m['zip5'].nunique()

3282

In [285]:
df_m.loc[df_m['FDXHD_unpadded_edd_dea'] != 'None']['zip5'].nunique()

3282

In [286]:
df_m.loc[df_m['FDXHD_unpadded_edd_dea'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date


In [287]:
dea.loc[dea['zip5'].isin(df_m.loc[df_m['FDXHD_unpadded_edd_dea'] == 'None']['zip5'])].head()

,zip5,unpadded_edd,carrier_code,package_count,unpadded_edd_dea_count


In [288]:
df_m.loc[df_m['ONTRGD_unpadded_edd_dea'] != 'None']['zip5'].nunique()

3156

In [289]:
df_m.loc[df_m['ONTRGD_unpadded_edd_dea'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
1117,37160,1,0.015209,37.571429,0.954373,None,None,263.0,None,0.05864,0.08,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
1713,33030,1,0.005435,26.285714,0.989848,None,None,197.0,None,0.053662,0.04,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
1742,53226,1,0.054945,26.000000,0.989130,None,None,184.0,None,0.062854,0.07,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
2377,45223,1,0.005263,27.142857,0.983607,None,None,183.0,None,0.086171,0.11,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
2648,78261,1,0.029915,33.428571,0.995392,None,None,217.0,None,0.05024,0.04,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13


In [290]:
dea.loc[dea['zip5'].isin(df_m.loc[df_m['ONTRGD_unpadded_edd_dea'] == 'None']['zip5'])].sort_values(['zip5','unpadded_edd'])[0:100]

,zip5,unpadded_edd,carrier_code,package_count,unpadded_edd_dea_count
98300,30121,2025-11-01,FDXHD,21,21
98301,30121,2025-11-02,FDXHD,29,29
98302,30121,2025-11-02,ONTRGD,2,1
98303,30121,2025-11-03,FDXHD,38,38
98304,30121,2025-11-04,FDXHD,31,31
98305,30121,2025-11-05,FDXHD,20,20
98306,30121,2025-11-06,FDXHD,29,25
98307,30121,2025-11-06,ONTRGD,1,1
98308,30121,2025-11-07,FDXHD,25,25
98309,30121,2025-11-08,FDXHD,37,37


In [291]:
df_m.loc[df_m['zip5'] == '07444'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date


In [292]:
dea.loc[dea['zip5'] == '07444']

,zip5,unpadded_edd,carrier_code,package_count,unpadded_edd_dea_count
23089,07444,2025-11-01,FDXHD,21,21
23090,07444,2025-11-02,FDXHD,29,29
23091,07444,2025-11-03,FDXHD,26,26
23092,07444,2025-11-03,ONTRGD,1,1
23093,07444,2025-11-04,FDXHD,23,22
23094,07444,2025-11-05,FDXHD,15,15
23095,07444,2025-11-06,FDXHD,19,18
23096,07444,2025-11-06,ONTRGD,2,2
23097,07444,2025-11-07,FDXHD,15,15
23098,07444,2025-11-08,FDXHD,23,23


### NA Backlog Zips

In [293]:
backlog_df = read_helper(
        './data/execution_data/backlog',
        start_date='2025-10-31',
        end_date='2025-11-13',
        date_col_name='date'
        )
backlog_df.head(2)

,dest_facility_num,zip5,days_behind,carrier_code,date
0,928,92832,0.043658,FDXHD,2025-10-31
1,468,45894,0.052419,FDXHD,2025-10-31


In [294]:
backlog_df.groupby(['carrier_code','date'])['zip5'].nunique()

carrier_code  date      
FDXHD         2025-10-31    86101
              2025-11-01    86178
              2025-11-02    86178
              2025-11-03    86178
              2025-11-04    86178
              2025-11-05    86114
              2025-11-07    86191
              2025-11-08    86114
              2025-11-09    86114
              2025-11-10    86114
              2025-11-12    86114
              2025-11-13    86114
ONTRGD        2025-10-31    11373
              2025-11-03    11373
              2025-11-04    11373
              2025-11-05    11373
              2025-11-06    11373
              2025-11-07    11373
              2025-11-10    11373
              2025-11-11    11373
              2025-11-12    11373
              2025-11-13    11373
Name: zip5, dtype: int64

In [295]:
df_m['zip5'].nunique()

3282

In [296]:
df_m.loc[df_m['FDXHD_days_behind'] != 'None']['zip5'].nunique()

2974

In [297]:
df_m.loc[df_m['FDXHD_days_behind'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
186,77007,1,0.147601,77.428571,0.989091,0.982759,ok,550.0,58.0,None,0.02,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
196,77018,1,0.116972,62.285714,0.995575,0.979592,ok,452.0,49.0,None,0.02,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
202,77024,1,0.110849,60.571429,0.988814,1.0,ok,447.0,41.0,None,0.02,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
221,75022,1,0.037879,37.714286,0.996364,1.0,ok,275.0,16.0,None,0.04,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
225,75028,1,0.035211,60.857143,0.993435,0.944444,ok,457.0,18.0,None,0.04,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13


In [298]:
backlog_df.loc[backlog_df['zip5'] == '77007']

,dest_facility_num,zip5,days_behind,carrier_code,date
81606,6030,77007,0.02,ONTRGD,2025-10-31
32607,6030,77007,0.02,ONTRGD,2025-11-03
86974,6030,77007,0.02,ONTRGD,2025-11-04
94471,6030,77007,0.03,ONTRGD,2025-11-05
10027,6030,77007,0.03,ONTRGD,2025-11-06
58888,6030,77007,0.06,ONTRGD,2025-11-07
44526,6030,77007,0.02,ONTRGD,2025-11-10
8402,6030,77007,0.01,ONTRGD,2025-11-11
51624,6030,77007,0.01,ONTRGD,2025-11-12
65739,6030,77007,0.02,ONTRGD,2025-11-13


In [252]:
df_m.loc[df_m['ONTRGD_days_behind'] != 'None']['zip5'].nunique()

3605

In [253]:
df_m.loc[df_m['ONTRGD_days_behind'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date


In [254]:
backlog_df

,dest_facility_num,zip5,days_behind,carrier_code,date
0,993,97894,0.029614,FDXHD,2025-10-31
1,323,31728,0.076963,FDXHD,2025-10-31
2,3101,11605,0.040464,FDXHD,2025-10-31
3,745,74599,0.080460,FDXHD,2025-10-31
4,691,68813,0.075032,FDXHD,2025-10-31
...,...,...,...,...,...
35307,688,68842,0.026197,FDXHD,2025-11-12
35308,701,70399,0.043045,FDXHD,2025-11-12
35309,930,93166,0.080195,FDXHD,2025-11-12
35310,701,70308,0.043045,FDXHD,2025-11-12
